#### 온라인 선호지수 도출하는 함수

#### [달라지는 변수]
#### 1)sampling 개수 from customer segmetation : 10000명, 20000명
#### 2)distance 종류 : cosine, euclidean
#### 3)kmeans clustering 적용 or not

In [1]:
import numpy as np
import pandas as pd
import sklearn
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine, euclidean
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance

In [3]:
from sklearn.metrics import mean_squared_error
def get_mse(pred,actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)

# 1. 구매선호지수

## 1) 10000명 sampling

In [23]:
#데이터 불러오기 및 초기화
Purchase_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Purchase_Count.csv")
Purchase_Count = Purchase_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_10000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Purchase_Count = pd.merge(Purchase_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Purchase_Count['CLNT_ID'].as_matrix()
col_names = Purchase_Count.columns.values
col_names = col_names[1:]
PC = Purchase_Count.as_matrix()
PC = PC[:,1:]
Purchase_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Purchase_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 13.162443018558125
(5029, 37)
(2477, 37)


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


## 2) 20000명 sampling

In [28]:
#데이터 불러오기 및 초기화
Purchase_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Purchase_Count.csv")
Purchase_Count = Purchase_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_20000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Purchase_Count = pd.merge(Purchase_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Purchase_Count['CLNT_ID'].as_matrix()
col_names = Purchase_Count.columns.values
col_names = col_names[1:]
PC = Purchase_Count.as_matrix()
PC = PC[:,1:]
Purchase_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Purchase_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 14.349619166222743
(10265, 37)
(5057, 37)


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


### 구매선호지수 계산

In [25]:
###파라메터 = k랑 코사인 
def purchase_prediction(train_mat, test_mat, metric_, k, output = False, scaling = False):
    if scaling == True:
        train_mat = (train_mat.T/train_mat.sum(axis=1)).T
        test_mat = (test_mat.T/test_mat.sum(axis=1)).T
    
    dist_out = 1 - pairwise_distances(train_mat, metric= metric_)

    #without kmeans clustering
    user_pred = dist_out.dot(train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T
    print('w/o kmeans, train:  ',get_mse(user_pred, train_mat))
    print('w/o kmeans, test:  ',get_mse(user_pred, test_mat))

    #with kmeans clustering
    neigh = NearestNeighbors(k, metric_)
    neigh.fit(train_mat)
    top_k_distances, top_k_users = neigh.kneighbors(train_mat, return_distance = True)
    user_pred_k = np.zeros(train_mat.shape)
    
    for i in range(PC_train.shape[0]):
        user_pred_k[i,:] = top_k_distances[i].T.dot(train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T  
    online_pref = pd.DataFrame(user_pred_k)
    na_element_index = np.where(np.isnan(online_pref))
    na_row_index = np.unique(na_element_index[0])
    removed_CLNT_ID = PC_train.index.values[na_row_index]
    existed_CLNT_ID = np.delete(PC_train.index.values,na_row_index)
    online_pref2 = online_pref.dropna()
    user_pred_k2 = online_pref2.as_matrix()
    train_mat2 = train_mat
    exist_idx = np.delete([range(online_pref.shape[0])],na_row_index)
    train_mat2 = train_mat2[exist_idx,:]
    print('w/ kmeans, train:  ',get_mse(user_pred_k2, train_mat2))
    print('w/ kmeans, test:  ',get_mse(user_pred_k2, test_mat))

    #kmeans 적용한 선호지수 matrix 생성
    if output == True :
        col_names = Purchase_Count.columns.values
        row_names = existed_CLNT_ID
        Online_Preference_Count = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)
        return Online_Preference_Count

- 10000명 experiments

In [17]:
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "cosine", 5, True)

w/o kmeans, train:   86.29608098337549
w/o kmeans, test:   96.9784070077395


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   28.63931852286754
w/ kmeans, test:   119.46998898239328


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [36]:
#10000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "euclidean", 5, True)

w/o kmeans, train:   182.66721252039383
w/o kmeans, test:   204.19730497273184


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   28.63931852286754
w/ kmeans, test:   119.46998898239328


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [61]:
#10000명 sampling한 경우, correlation 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train:   85.40834048933255
w/o kmeans, test:   99.37528162986564


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   28.63931852286754
w/ kmeans, test:   119.46998898239328


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [26]:
##Scaling 적용할 경우
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "cosine", 5, True, True)
opc = purchase_prediction(train_mat,test_mat, "euclidean", 5, True, True)
opc = purchase_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train:   0.028131755848825844
w/o kmeans, test:   0.06975756911905921


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.002720327647331267
w/ kmeans, test:   0.08571259018474586
w/o kmeans, train:   0.04161443327691153
w/o kmeans, test:   0.08295957767369831


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.002720327647331267
w/ kmeans, test:   0.08571259018474586
w/o kmeans, train:   0.028816220429387387
w/o kmeans, test:   0.07564184275249344


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.002720327647331267
w/ kmeans, test:   0.08571259018474586


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


- 20000명 experiments

In [7]:
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "cosine", 5, True)

w/o kmeans, train:   408.05193415503004
w/o kmeans, test:   587.0903686780509


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   92.52537404004526
w/ kmeans, test:   611.035951966975


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [44]:
#20000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "euclidean", 5, True)

w/o kmeans, train:   754.4943007098875
w/o kmeans, test:   880.4058472057757


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   92.52537404004526
w/ kmeans, test:   611.035951966975


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [8]:
#20000명 sampling한 경우, correlation 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train:   551.052167205421
w/o kmeans, test:   592.0709278543959


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   92.52537404004526
w/ kmeans, test:   611.035951966975


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [29]:
##Scaling 적용할 경우
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
purchase_prediction(train_mat,test_mat, "cosine", 5, True, True)
purchase_prediction(train_mat,test_mat, "euclidean", 5, True, True)
purchase_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train:   0.02787429540003764
w/o kmeans, test:   0.0633956288578198


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.0021484541788169575
w/ kmeans, test:   0.07772765830475424
w/o kmeans, train:   0.04411056198193033
w/o kmeans, test:   0.07349003954176009


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.0021484541788169575
w/ kmeans, test:   0.07772765830475424
w/o kmeans, train:   0.03017625696720918
w/o kmeans, test:   0.06956324211410529


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.0021484541788169575
w/ kmeans, test:   0.07772765830475424


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
6382771,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018519,0.000000,0.685185,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
456508,0.777778,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5888892,0.072825,0.024841,0.000000,0.000000,0.000000,0.033262,0.016550,0.806235,0.000000,0.004064,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3399944,0.883867,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.116133,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5793121,0.058681,0.067825,0.519429,0.000000,0.000000,0.085000,0.251477,0.006970,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
840339,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.363636,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2161500,0.052976,0.432836,0.000000,0.000000,0.000000,0.045757,0.000000,0.425077,0.000000,0.027212,...,0.000000,0.000750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4257175,0.000000,0.000000,0.000000,0.000000,0.015971,0.081058,0.000000,0.000000,0.000000,0.009604,...,0.019209,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
94099,0.001848,0.003695,0.000000,0.027454,0.001848,0.039766,0.001848,0.000000,0.000000,0.792910,...,0.000000,0.001848,0.000000,0.003695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2893690,0.020571,0.024339,0.000000,0.000000,0.000000,0.016803,0.000000,0.032503,0.000000,0.326097,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010886,0.000000,0.000000,0.003577,0.000000


# 2. 검색선호지수

## 1)10000명 sampling

In [31]:
#데이터 불러오기 및 초기화
Search_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Search_CNT.csv")
Search_Count = Search_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_10000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Search_Count = pd.merge(Search_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Search_Count['CLNT_ID'].as_matrix()
col_names = Search_Count.columns.values
col_names = col_names[1:]
PC = Search_Count.as_matrix()
PC = PC[:,1:]
Search_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Search_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 10.11659141155544
(5029, 37)
(2477, 37)


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


## 2)20000명 sampling

In [34]:
#데이터 불러오기 및 초기화
Search_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Search_CNT.csv")
Search_Count = Search_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_20000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Search_Count = pd.merge(Search_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Search_Count['CLNT_ID'].as_matrix()
col_names = Search_Count.columns.values
col_names = col_names[1:]
PC = Search_Count.as_matrix()
PC = PC[:,1:]
Search_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Search_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 10.898478428826948
(10265, 37)
(5057, 37)


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


### 검색선호지수 계산

In [32]:
###파라메터 = k랑 코사인 
def search_prediction(train_mat, test_mat, metric_, k, output = False, scaling = False):
    if scaling == True:
        train_mat = (train_mat.T/train_mat.sum(axis=1)).T
        test_mat = (test_mat.T/test_mat.sum(axis=1)).T
        
    dist_out = 1 - pairwise_distances(train_mat, metric= metric_)

    #without kmeans clustering
    user_pred = dist_out.dot(train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T
    print('w/o kmeans, train:  ',get_mse(user_pred, train_mat))
    print('w/o kmeans, test:  ',get_mse(user_pred, test_mat))

    #with kmeans clustering
    neigh = NearestNeighbors(k, metric_)
    neigh.fit(train_mat)
    top_k_distances, top_k_users = neigh.kneighbors(train_mat, return_distance = True)
    user_pred_k = np.zeros(train_mat.shape)
    
    for i in range(PC_train.shape[0]):
        user_pred_k[i,:] = top_k_distances[i].T.dot(train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T  
    online_pref = pd.DataFrame(user_pred_k)
    na_element_index = np.where(np.isnan(online_pref))
    na_row_index = np.unique(na_element_index[0])
    removed_CLNT_ID = PC_train.index.values[na_row_index]
    existed_CLNT_ID = np.delete(PC_train.index.values,na_row_index)
    online_pref2 = online_pref.dropna()
    user_pred_k2 = online_pref2.as_matrix()
    train_mat2 = train_mat
    exist_idx = np.delete([range(online_pref.shape[0])],na_row_index)
    train_mat2 = train_mat2[exist_idx,:]
    print('w/ kmeans, train:  ',get_mse(user_pred_k2, train_mat2))
    print('w/ kmeans, test:  ',get_mse(user_pred_k2, test_mat))

    #kmeans 적용한 선호지수 matrix 생성
    if output == True :
        col_names = Search_Count.columns.values
        row_names = existed_CLNT_ID
        Online_Preference_Search = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)
        return Online_Preference_Search

- 10000명 experiments

In [49]:
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True)

w/o kmeans, train:   4615.882115392347
w/o kmeans, test:   924.2527845270545


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   3011.175289814995
w/ kmeans, test:   1360.5331990912446


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [54]:
#10000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True)

w/o kmeans, train:   16327.345607006568
w/o kmeans, test:   16043.406573066946


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   3011.175289814995
w/ kmeans, test:   1360.5331990912446


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [11]:
#10000명 sampling한 경우, correlation 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train:   4598.456219440562
w/o kmeans, test:   957.5635999319992


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   3011.175289814995
w/ kmeans, test:   1360.5331990912446


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [33]:
##Scaling 적용할 경우
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True, True)
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True, True)
ops = search_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train:   0.03602239096595326
w/o kmeans, test:   0.12796806259553326


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.003323832338825208
w/ kmeans, test:   0.14392997683927655
w/o kmeans, train:   0.07715384905738594
w/o kmeans, test:   0.1634055741779153


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.003323832338825208
w/ kmeans, test:   0.14392997683927655
w/o kmeans, train:   0.042150909706843716
w/o kmeans, test:   0.13901195661203594


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.003323832338825208
w/ kmeans, test:   0.14392997683927655


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


- 20000명 experiments

In [12]:
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True)

w/o kmeans, train:   4615.882115392347
w/o kmeans, test:   924.2527845270545


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   3011.175289814995
w/ kmeans, test:   1360.5331990912446


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [14]:
#20000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True)

w/o kmeans, train:   16327.345607006568
w/o kmeans, test:   16043.406573066946


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   3011.175289814995
w/ kmeans, test:   1360.5331990912446


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [15]:
#20000명 sampling한 경우, correlation 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train:   4598.456219440562
w/o kmeans, test:   957.5635999319992


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   3011.175289814995
w/ kmeans, test:   1360.5331990912446


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [35]:
##Scaling 적용할 경우
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True, True)
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True, True)
ops = search_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train:   0.03581917196583815
w/o kmeans, test:   0.11192931135624402


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.002683239924003692
w/ kmeans, test:   0.1290034655472081
w/o kmeans, train:   0.07827259425732647
w/o kmeans, test:   0.1387689249691853


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


w/ kmeans, train:   0.002683239924003692
w/ kmeans, test:   0.1290034655472081
w/o kmeans, train:   0.044407876584194365
w/o kmeans, test:   0.12250626694137244


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.002683239924003692
w/ kmeans, test:   0.1290034655472081


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
